In [1]:
import torch
import matplotlib.pyplot as plt
import torchvision

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [2]:
torchvision.datasets.CIFAR10(root="./data", download=True, train=True)

Files already downloaded and verified


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train

In [3]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

def plot_sample(X, y, index):
    plt.figure(figsize = (15,2))
    plt.imshow(X[index])
    plt.xlabel(classes[y[index]])


In [110]:
from torch import nn
import torch.nn.functional as F

IMG_SIZE=32
NUM_CLASSES = 10
TRAINING_BATCH_SIZE = 100
TESTING_BATCH_SIZE = 10

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        print("Init:")
        self.cn1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        
        x = torch.randn(3, IMG_SIZE, IMG_SIZE).view(-1, 3, IMG_SIZE, IMG_SIZE)
        print(x.shape)
        self._to_linear = None
        self.convs(x)
        
        print(x.shape)
        print(self._to_linear)
        
        self.fc1 = nn.Linear(self._to_linear, NUM_CLASSES)
        
    def convs(self, x):
        # print(x.shape)
        assert x.shape[1] == 3
        
        x = F.max_pool2d(F.relu(self.cn1(x)), (2, 2))
        
        self._to_linear = x.flatten(start_dim=1).shape[1]
        
        return x
    
    def forward(self, x):
        # print("Forwarding:")
        # print(x.shape)
        
        x = self.convs(x)        
        x = x.view(-1, self._to_linear)

        x = self.fc1(x)
        
        return x
    
Net()

Init:
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
7200


Net(
  (cn1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=7200, out_features=10, bias=True)
)

In [111]:

import os
import time
import random
from tqdm import tqdm

class Learning():
    def __init__(self, model, optimizer, loss_fn, training_dataloader, testing_dataloader, img_size: int, epochs: int):
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.epochs = epochs
        self.training_dataloader = training_dataloader
        self.testing_dataloader = testing_dataloader
        self.testing_batch = list(iter(testing_dataloader))
        self.img_size = img_size
        
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.log_path = f"/data/logs/{int(time.time())}.csv"

    def train(self):
        for epoch in range(self.epochs):
            for i, (batch_X, batch_y) in enumerate(tqdm(iter(self.training_dataloader), desc="Epoch " + str(epoch + 1))):
                # print("Training:")
                # print(batch_y.shape)
                batch_X = (batch_X / 255).to(dtype=torch.float32)
                batch_y = F.one_hot(batch_y.long(), num_classes=NUM_CLASSES).to(dtype=torch.float32)
                
                acc, loss = self._fwd_pass(batch_X, batch_y, train=True)
                val_acc, val_loss = self.test()
                self.log_to_file(str(epoch) + '-' + str(i), acc, loss, val_acc, val_loss)

            print("Acc:", acc)
            print("Loss:", loss)


    def test(self):
        i = random.randint(0, len(self.testing_batch) - 1)
        batch_X, batch_y = self.testing_batch[i]
        
        batch_X = (batch_X / 255.0).to(dtype=torch.float32).view(-1, 3, self.img_size, self.img_size)
        batch_y = F.one_hot(batch_y.long(), num_classes=NUM_CLASSES).to(dtype=torch.float32)
        
        acc, loss = self._fwd_pass(batch_X, batch_y, train=False)

        return acc, loss
    

    def _fwd_pass(self, X, y, train=False):
        if train:
            self.model.zero_grad()
                
        output = self.model(X)
        # print(y.shape)
        # print(output.shape)
        # print(y.dtype)
        # print(output.dtype)
        # print(output[0])
        loss = self.loss_fn(y, output)

        n_matches = torch.count_nonzero(torch.argmax(output, dim=1) == torch.argmax(y, dim=1)).item()
        
        if train:
            loss.backward()
            self.optimizer.step()
            
        return round(n_matches / len(X), 2), loss
    
    def log_to_file(self, id: str, train_acc, train_loss, val_acc, val_loss):
        append = f"{id},{train_acc},{train_loss},{val_acc},{val_loss}\n"
        full_path = os.path.abspath(os.getcwd()) + self.log_path
        
        with open(full_path, "a") as f:
            if os.path.getsize(full_path) <= 0:
                f.write("id,train_acc,train_loss,val_acc,val_loss\n")
                
            f.write(append)
            
            
from torchvision import transforms
from torch import optim

training_transforms = transforms.Compose([
    transforms.ToTensor(),
])

testing_transforms = transforms.Compose([
    transforms.ToTensor(),
])

training_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, transform=training_transforms)
training_dataloader = torch.utils.data.DataLoader(training_dataset, shuffle=True, batch_size=TRAINING_BATCH_SIZE)

testing_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, transform=testing_transforms)
testing_dataloader = torch.utils.data.DataLoader(testing_dataset, shuffle=True, batch_size=TESTING_BATCH_SIZE)

# index = 1
# plt.figure(figsize = (15,2))
# plt.imshow(training_dataset[index][0])
# plt.xlabel(classes[training_dataset[index][1]])

In [113]:

net = Net()

learning = Learning(
    model=net,
    optimizer=optim.Adam(net.parameters(), lr=0.001),
    loss_fn=nn.MSELoss(),
    img_size=IMG_SIZE,
    training_dataloader=training_dataloader,
    testing_dataloader=testing_dataloader,
    epochs=3
)

learning.train()

Init:
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
7200


Epoch 2:   1%|          | 3/500 [00:00<00:18, 27.33it/s]

Acc: 0.29
Loss: tensor(0.0850, grad_fn=<MseLossBackward>)


Epoch 3:   1%|          | 3/500 [00:00<00:18, 27.54it/s]

Acc: 0.25
Loss: tensor(0.0840, grad_fn=<MseLossBackward>)


Epoch 3: 100%|██████████| 500/500 [00:17<00:00, 28.64it/s]

Acc: 0.36
Loss: tensor(0.0795, grad_fn=<MseLossBackward>)
